# Crypto Forecasting

Previous "kompetenseftermiddag" found on [kaggle.com/lundet](https://www.kaggle.com/lundet/kompetenseftermiddag-crypto-forecasting/).  

At the previous meeting we had a lot of theory, discussions and just little implementation. This "kompetenseftermiddag" (competence afternoon) we'll implement a bunch more! 

Some of the stuff we went through:
- Time Series theory like composition
    - Time Series are often decomposed into _Seasonality_, _Trend_, _Noise_ and sometimes even _Auto-Correlation_.
- Techniques to approach Time Series
    - This includes models and other things
- How Fourier Transform can impact... seasonality, or what did we say again?
    - For the ones who missed last meeting we had a huge discussion on FFT :)))))

But for now we'll look at
1. Wrangling Data
2. Building Models
3. Measuring Performance

Because people usually run a fresh run on `Collaboratory` I won't dive into building our `venv`.

## Imports
First of all we need some standard imports, I'll add `pandas` and `numpy` which are commonly used for math (on the CPU), data wrangling/visualization and I/O.

<details>
<summary>Wanna know a little more about `pandas`?</summary>
Pandas 
</details>

In [1]:
from IPython.display import clear_output
!pip install pandas-datareader
clear_output()

In [2]:
import pandas as pd
import pandas_datareader as pdr
import numpy as nd
from datetime import datetime


ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

### Minor Footnote

A few weeks ago I impressed my client a lot.

(GIF OF JAW DROP)

Using `arrow` format which uses "Column Compression". 

(IMAGE OF COLUMN COMPRESSION)

I took our files from 270MB -> 5MB loosing 0 data points.

In [ ]:
# Add pandas dataset library
btc_data = pdr.get_data_yahoo(['BTC-USD'], 
                          start=datetime.datetime(2018, 1, 1), 
                          end=datetime.datetime(2020, 12, 2))['Close']
btc_data.head()